### Desafio 2
  - Lauro, Rodrigo (ing.rodrigo.lauro@gmail.com)
  - Nº SIU a2120



In [1]:
!pip install -U gensim

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import multiprocessing
from gensim.models import Word2Vec

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 68.5 MB/s eta 0:00:00


### Datos
Utilizaremos como dataset canciones de bandas de habla inglesa.

In [2]:
# Descargar la carpeta de dataset
import os
import platform
if os.access('./songs_dataset', os.F_OK) is False:
    if os.access('songs_dataset.zip', os.F_OK) is False:
        if platform.system() == 'Windows':
            !curl https://raw.githubusercontent.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/main/datasets/songs_dataset.zip -o songs_dataset.zip
        else:
            !wget songs_dataset.zip https://github.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/raw/main/datasets/songs_dataset.zip
    !unzip -q songs_dataset.zip
else:
    print("El dataset ya se encuentra descargado")

--2025-11-04 19:12:54--  http://songs_dataset.zip/
Resolving songs_dataset.zip (songs_dataset.zip)... failed: Name or service not known.
wget: unable to resolve host address ‘songs_dataset.zip’
--2025-11-04 19:12:54--  https://github.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/raw/main/datasets/songs_dataset.zip
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/main/datasets/songs_dataset.zip [following]
--2025-11-04 19:12:54--  https://raw.githubusercontent.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/main/datasets/songs_dataset.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.

### Alumno

- Crear sus propios vectores con Gensim basado en lo visto en clase con otro dataset.
- Probar términos de interés y explicar similitudes en el espacio de embeddings (sacar conclusiones entre palabras similitudes y diferencias).
- Graficarlos.
- Obtener conclusiones.

### Crear vectores con Gensim

In [3]:
import pandas as pd

df_alumno = pd.read_csv('songs_dataset/nirvana.txt', sep='/n', header=None)

print("Cantidad de documentos:", df_alumno.shape[0])

# muestro alguna lineas de las canciones
df_alumno.head(10)



Cantidad de documentos: 1985


/tmp/ipython-input-2264429251.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df_alumno = pd.read_csv('songs_dataset/nirvana.txt', sep='/n', header=None)


,0
0,"Load up on guns, bring your friends"
1,It's fun to lose and to pretend
2,She's over-bored and self-assured
3,"Oh no, I know a dirty word Hello, hello, hello..."
4,"Hello, hello, hello, how low"
5,"Hello, hello, hello, how low"
6,"Hello, hello, hello With the lights out, it's ..."
7,"Here we are now, entertain us"
8,I feel stupid and contagious
9,"Here we are now, entertain us"


In [4]:

#preoproc
from tensorflow.keras.preprocessing.text import text_to_word_sequence

sentence_tokens = []

# recorro todas las filas y transformo las oraciones en palabras
for _, row in df_alumno[:None].iterrows():
    sentence_tokens.append(text_to_word_sequence(row[0]))

print("Primeras dos oraciones", sentence_tokens[:2])


#creo los vectores (word2vec)
from gensim.models.callbacks import CallbackAny2Vec


# sobrecargo el callback para poder tobtener el "loss"
class callback(CallbackAny2Vec):
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss


# Crearmos el modelo generador de vectores
# tengo pocos datos:
  # achico el min_count
  # se aumenta el parámetro window a 8, ya que en las letras de canciones
  # las frases suelen tener alrededor de 8 palabras.
  # esto permite capturar mejor las relaciones dentro de un mismo verso.

w2v_model = Word2Vec(min_count=2,     # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=8,        # cant de palabras antes y desp de la predicha
                     vector_size=300, # dimensionalidad de los vectores
                     negative=20,     # cantidad de negative samples
                     workers=1,
                     sg=1)            # modelo 0:CBOW  1:skipgram



# obtengo el vocabulario con los tokens
w2v_model.build_vocab(sentence_tokens)

print("Cantidad de docs en el corpus:", w2v_model.corpus_count)

print("Cantidad de words distintas en el corpus:", len(w2v_model.wv.index_to_key))


#3 - Entrenar embeddings

# entreno el modelo generador de vectores
# utilizo nuestro callback
# 55 epocas parece ser el punto optimo de entrenamiento (evita sobreajuste)
w2v_model.train(sentence_tokens,
                 total_examples=w2v_model.corpus_count,
                 epochs=55,
                 compute_loss = True,
                 callbacks=[callback()]
                 )


Primeras dos oraciones [['load', 'up', 'on', 'guns', 'bring', 'your', 'friends'], ["it's", 'fun', 'to', 'lose', 'and', 'to', 'pretend']]
Cantidad de docs en el corpus: 1985
Cantidad de words distintas en el corpus: 898
Loss after epoch 0: 197429.0
Loss after epoch 1: 117083.15625
Loss after epoch 2: 113756.0
Loss after epoch 3: 113922.84375
Loss after epoch 4: 109062.375
Loss after epoch 5: 105253.0
Loss after epoch 6: 103012.6875
Loss after epoch 7: 98385.25
Loss after epoch 8: 94055.4375
Loss after epoch 9: 82465.625
Loss after epoch 10: 79018.25
Loss after epoch 11: 76584.875
Loss after epoch 12: 74749.75
Loss after epoch 13: 71081.5
Loss after epoch 14: 71448.625
Loss after epoch 15: 69032.875
Loss after epoch 16: 65420.5
Loss after epoch 17: 65091.75
Loss after epoch 18: 64008.75
Loss after epoch 19: 63531.5
Loss after epoch 20: 61913.125
Loss after epoch 21: 60523.25
Loss after epoch 22: 61159.375
Loss after epoch 23: 60979.25
Loss after epoch 24: 57562.5
Loss after epoch 25: 555

(448180, 679030)

### Probar términos de interés y explicar similitudes

In [5]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["smell"], topn=10)

[('superfly', 0.8004654049873352),
 ('sex', 0.8000050187110901),
 ('lemonade', 0.798118531703949),
 ('candy', 0.7903543710708618),
 ('disco', 0.7077271938323975),
 ('promise', 0.6784499287605286),
 ('fall', 0.6636168956756592),
 ('tell', 0.6312507390975952),
 ('pie', 0.6294639110565186),
 ('security', 0.6284490823745728)]

In [6]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["contagious"], topn=10)

[('stupid', 0.9627507328987122),
 ('blessed', 0.813519299030304),
 ('gift', 0.8059986233711243),
 ('dawn', 0.7549125552177429),
 ('fades', 0.7444480061531067),
 ('gray', 0.7388702034950256),
 ('guilty', 0.7307881116867065),
 ('buzz', 0.7109196782112122),
 ('feel', 0.6893523931503296),
 ('pretty', 0.6890068650245667)]

In [7]:
w2v_model.wv.most_similar(positive=["albino"], topn=10)

[('mosquito', 0.9989333748817444),
 ('mulatto', 0.9985504150390625),
 ('libido', 0.9677892923355103),
 ('wear', 0.9134306311607361),
 ('shield', 0.8837183713912964),
 ("everything's", 0.7969343662261963),
 ('fault', 0.7936939597129822),
 ('daze', 0.7619130611419678),
 ('trend', 0.7448481321334839),
 ('candles', 0.736147403717041)]

### Graficarlos

In [10]:
from sklearn.decomposition import IncrementalPCA
from sklearn.manifold import TSNE
import numpy as np

def reduce_dimensions_pca(model, num_dimensions = 2 ):

    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    return vectors, labels


def reduce_dimensions_tsne(model, n_components=3, perplexity=30, random_state=42):

    X = model.wv.vectors
    labels = model.wv.index_to_key

    tsne = TSNE(
        n_components=n_components,
        perplexity=perplexity,
        random_state=random_state,
        init='pca',
        learning_rate='auto'
    )

    X_reduced = tsne.fit_transform(X)
    return X_reduced, labels

In [11]:
# Graficar los embedddings en 3D
import plotly.graph_objects as go
import plotly.express as px

MAX_WORDS=200

#puede graficarse utilizando PCA.
#elijo hacerlo con T-sne
#vecs, labels = reduce_dimensions(w2v_model,3)
vecs, labels = reduce_dimensions_tsne(w2v_model, 3)

fig = px.scatter_3d(x=vecs[:MAX_WORDS,0]
                    , y=vecs[:MAX_WORDS,1]
                    , z=vecs[:MAX_WORDS,2]
                    ,text=labels[:MAX_WORDS])

fig.update_traces(marker_size = 3)
fig.show(renderer="colab")

Reduciendo dimensiones con t-SNE, esto puede demorar unos minutos...


### Obtener conclusiones.

  * El modelo Word2Vec (Skip-Gram) entrenado sobre letras de Nirvana logró capturar relaciones semánticas coherentes dentro del vocabulario característico de las canciones.
  * Palabras como smell, spirit, mind o pain aparecen asociadas en el espacio vectorial, lo cual refleja cierta comprensión temática y contextual del corpus.

  * Durante el entrenamiento se observó que el loss disminuye hasta la época 55, punto a partir del cual comienza a aumentar, lo que sugiere sobreajuste. Por ello, se consideró ese punto como óptimo.
  * El parámetro window=8 se ajustó en función de la estructura típica de las letras musicales, que suelen tener frases de longitud   de 8 palabras , permitiendo al modelo capturar relaciones de contexto dentro de un mismo verso.
  * Si bien el corpus es relativamente pequeño, los embeddings generados muestran asociaciones semánticas consistentes. Sería interesante repetir el experimento con un corpus más extenso o combinando múltiples artistas para mejorar la densidad y generalización del espacio vectorial.

  * La visualización con t-SNE permitire explorar visualmente estas relaciones y validar la proximidad entre palabras afines.